# Ipsilateral-GTV Model Pipeline


## Environment setup

In [36]:
from DicomRTTool.ReaderWriter import DicomReaderWriter, ROIAssociationClass
import SimpleITK as sitk
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, ConcatDataset, TensorDataset
import pandas as pd

## Data preparation

### **Enter path to dicom folder here

In [37]:
Dicom_path = r'/home/lam3654/MSAI_pneumonitis/data/BATCH 1'

In [38]:
def display_slices(image, mask, skip=1):
    """
    Displays a series of slices in z-direction that contains the segmented regions of interest.
    Ensures all contours are displayed in consistent and different colors.
        Parameters:
            image (array-like): Numpy array of image.
            mask (array-like): Numpy array of mask.
            skip (int): Only print every nth slice, i.e. if 3 only print every 3rd slice, default 1.
        Returns:
            None (series of in-line plots).
    """

    slice_locations = np.unique(np.where(mask != 0)[0]) # get indexes for where there is a contour present 
    slice_start = slice_locations[0] # first slice of contour 
    slice_end = slice_locations[len(slice_locations)-1] # last slice of contour
    
    counter = 1
    
    for img_arr, contour_arr in zip(image[slice_start:slice_end+1], mask[slice_start:slice_end+1]): # plot the slices with contours overlayed ontop
        if counter % skip == 0: # if current slice is divisible by desired skip amount 
            masked_contour_arr = np.ma.masked_where(contour_arr == 0, contour_arr)
            plt.imshow(img_arr, cmap='gray', interpolation='none')
            plt.imshow(masked_contour_arr, cmap='cool', interpolation='none', alpha=0.5, vmin = 1, vmax = np.amax(mask)) # vmax is set as total number of contours so same colors can be displayed for each slice
            plt.show()
        counter += 1

In [ ]:
Dicom_left_reader = DicomReaderWriter(description='Examples', arg_max=True)
Dicom_left_reader.walk_through_folders(Dicom_path) # This will parse through all DICOM present in the folder and subfolders

In [39]:


Dicom_right_reader = DicomReaderWriter(description='Examples', arg_max=True)
Dicom_right_reader.walk_through_folders(Dicom_path) # This will parse through all DICOM present in the folder and subfolders


Loading through DICOM files:   0%|                                                                                                       | 0/204 [00:00<?, ?it/s]

Loading from /home/lam3654/MSAI_pneumonitis/data/BATCH 1/NU_0011/2019-06__Studies/NU.0011_NU.0011_RTst_2019-06-11_160923_SBRT.Planning.Scan_AY_n1__00000
Loading from /home/lam3654/MSAI_pneumonitis/data/BATCH 1/NU_0033/2018-06__Studies/NU.0033_NU.0033_RTDOSE_2018-06-21_152956_sbrt.lung.planning.w.contrast_NU.0033_n1__00000
Loading from /home/lam3654/MSAI_pneumonitis/data/BATCH 1/NU_0037/2012-09__Studies/NU.0037_NU.0037_RTDOSE_2012-09-12_091234_SBRT.planning.bellows_NU.0037_n1__00000
Loading from /home/lam3654/MSAI_pneumonitis/data/BATCH 1/NU_0019/2018-07__Studies/NU.0019_NU.0019_RTDOSE_2018-07-11_162227_sbrt.lung.planning_NU.0019_n1__00000
Loading from /home/lam3654/MSAI_pneumonitis/data/BATCH 1/NU_0011/2019-06__Studies/NU.0011_NU.0011_RTDOSE_2019-06-11_160923_MinIP.SBRT.Planning.Scan_._n1__00000
Loading from /home/lam3654/MSAI_pneumonitis/data/BATCH 1/NU_0004/2016-07__Studies/NU.0004_NU.0004_CT_2016-07-03_145128_SBRT.Lung.planning_NU.0004_n187__00000
Loading from /home/lam3654/MSAI_pne

Loading through DICOM files:   6%|█████▌                                                                                        | 12/204 [00:00<00:18, 10.33it/s]

Loading from /home/lam3654/MSAI_pneumonitis/data/BATCH 1/NU_0055/2016-04__Studies/NU.0055_NU.0055_RTst_2016-04-18_135434_SBRT.lung.planning_AY_n1__00000
Loading from /home/lam3654/MSAI_pneumonitis/data/BATCH 1/NU_0038/2012-03__Studies/NU.0038_NU.0038_CT_2012-03-05_113831_SBRT.Planning.Scan_NU.0038_n162__00000
Loading from /home/lam3654/MSAI_pneumonitis/data/BATCH 1/NU_0055/2016-04__Studies/NU.0055_NU.0055_RTPLAN_2016-04-18_135434_SBRT.lung.planning_NU.0055_n1__00000
Loading from /home/lam3654/MSAI_pneumonitis/data/BATCH 1/NU_0038/2012-03__Studies/NU.0038_NU.0038_RTPLAN_2012-03-05_113831_SBRT.Planning.Scan_NU.0038_n1__00000
Loading from /home/lam3654/MSAI_pneumonitis/data/BATCH 1/NU_0055/2016-04__Studies/NU.0055_NU.0055_CT_2016-04-18_135434_SBRT.lung.planning_NU.0055_n164__00000
Loading from /home/lam3654/MSAI_pneumonitis/data/BATCH 1/NU_0038/2012-03__Studies/NU.0038_NU.0038_RTst_2012-03-05_113831_SBRT.Planning.Scan_AY_n1__00000
Loading from /home/lam3654/MSAI_pneumonitis/data/BATCH 1/N

Loading through DICOM files:   7%|██████▍                                                                                       | 14/204 [00:00<00:20,  9.19it/s]

...

Loading from /home/lam3654/MSAI_pneumonitis/data/BATCH 1/NU_0006/2018-07__Studies/NU.0006_NU.0006_RTPLAN_2018-07-17_141739_MInIP.SBRT.lung.Bellows.Scan_NU.0006_n1__00000


Loading through DICOM files:  52%|████████████████████████████████████████████████▊                                            | 107/204 [00:15<00:36,  2.67it/s]

Had an error loading /home/lam3654/MSAI_pneumonitis/data/BATCH 1/NU_0005/2018-08__Studies/NU.0005_NU.0005_RTPLAN_2018-08-29_144651_SBRT.LUL.Plan.Scan.and.Bellows_._n1__00000/2.16.840.1.114362.1.12046989.25391989775.615974270.1010.1.dcm
Loading from /home/lam3654/MSAI_pneumonitis/data/BATCH 1/NU_0006/2018-07__Studies/NU.0006_NU.0006_RTDOSE_2018-07-17_141739_MInIP.SBRT.lung.Bellows.Scan_NU.0006_n1__00000
Had an error loading /home/lam3654/MSAI_pneumonitis/data/BATCH 1/NU_0030/2018-10__Studies/NU.0030_NU.0030_RTPLAN_2018-10-02_150021_MinIP.SBRT.Chest.Planning.Scan_NU.0030_n1__00000/2.16.840.1.114362.1.11940992.22933840372.569057948.717.549.dcm


Loading through DICOM files:  53%|█████████████████████████████████████████████████▋                                           | 109/204 [00:16<00:33,  2.87it/s]

Loading from /home/lam3654/MSAI_pneumonitis/data/BATCH 1/NU_0034/2012-08__Studies/NU.0034_NU.0034_RTst_2012-08-30_110406_SBRT.planning.+.bellows_AY_n1__00000
Loading from /home/lam3654/MSAI_pneumonitis/data/BATCH 1/NU_0034/2012-08__Studies/NU.0034_NU.0034_RTDOSE_2012-08-30_110406_SBRT.planning.+.bellows_NU.0034_n1__00000
Loading from /home/lam3654/MSAI_pneumonitis/data/BATCH 1/NU_0034/2012-08__Studies/NU.0034_NU.0034_CT_2012-08-30_110406_SBRT.planning.+.bellows_NU.0034_n148__00000
Had an error loading /home/lam3654/MSAI_pneumonitis/data/BATCH 1/NU_0001/2018-10__Studies/NU.0001_NU.0001_RTPLAN_2018-10-08_144305_SBRT.Planning_NU.0001_n1__00000/2.16.840.1.114362.1.11940992.22933840372.569057282.346.7761.dcm
Loading from /home/lam3654/MSAI_pneumonitis/data/BATCH 1/NU_0034/2012-08__Studies/NU.0034_NU.0034_CT_2012-08-30_110406_SBRT.planning.+.bellows_NU.0034_n148__00000
Had an error loading /home/lam3654/MSAI_pneumonitis/data/BATCH 1/NU_0001/2018-10__Studies/NU.0001_NU.0001_RTPLAN_2018-10-08_

In [ ]:
all_rois = Dicom_left_reader.return_rois(print_rois=True) # Return a list of all rois present

In [ ]:
Contour_left_names = ['l lung-gtv']
Dicom_left_reader.set_contour_names_and_associations(contour_names=Contour_left_names)
Contour_right_names = ['r lung-gtv']
Dicom_right_reader.set_contour_names_and_associations(contour_names=Contour_right_names)

In [ ]:
left_indexes = Dicom_left_reader.which_indexes_have_all_rois()
right_indexes = Dicom_right_reader.which_indexes_have_all_rois()

In [ ]:
left_pt_indx = left_indexes[-1]
Dicom_left_reader.set_index(left_pt_indx)  # This index has all the structures, corresponds to pre-RT T1-w image for patient 011
Dicom_left_reader.get_images_and_mask()  # Load up the images and mask for the requested index

right_pt_indx = right_indexes[-1]
Dicom_right_reader.set_index(right_pt_indx)  # This index has all the structures, corresponds to pre-RT T1-w image for patient 011
Dicom_right_reader.get_images_and_mask()  # Load up the images and mask for the requested index

In [ ]:
left_image = Dicom_left_reader.ArrayDicom # image array
left_mask = Dicom_left_reader.mask # mask array
right_image = Dicom_right_reader.ArrayDicom # image array
right_mask = Dicom_right_reader.mask # mask array

In [ ]:
n_slices_skip = 4
display_slices(left_image, left_mask, skip = n_slices_skip) # visualize that our segmentations were succesfully convereted

In [ ]:
# Load the labels from the Excel file
labels_df = pd.read_excel('label.xlsx')

class CTMaskDataset(Dataset):
    def __init__(self, image, mask, labels_df):
        self.image = image
        self.mask = mask
        self.labels_df = labels_df

    def __len__(self):
        return len(self.image)

    def __getitem__(self, idx):
        img_slice = self.image[idx]
        mask_slice = self.mask[idx]

        # Normalize the image slice
        img_slice = (img_slice - img_slice.min()) / (img_slice.max() - img_slice.min())
        img_slice = torch.tensor(img_slice[np.newaxis], dtype=torch.float32)

        # Convert mask into binary format (0 for background, 1 for lungs)
        mask_slice = np.where(mask_slice > 0, 1, 0)
        mask_slice = torch.tensor(mask_slice, dtype=torch.long)

        # Get the corresponding label from the Excel file
        anon_id = self.labels_df.iloc[idx]['anon_id']  # Assuming 'anon_id' is the column name for the anonymous id
        label = self.labels_df.loc[self.labels_df['anon_id'] == anon_id, 'pneumonitis'].values[0]  # Assuming 'label' is the column name for the labels

        return img_slice, mask_slice, label

# Create the dataset and data loader
left_dataset = CTMaskDataset(left_image, left_mask, labels_df)
right_dataset = CTMaskDataset(right_image, right_mask, labels_df)
dataset = ConcatDataset([left_dataset, right_dataset])
data_loader = DataLoader(dataset, batch_size=4, shuffle=True)

## Define Model Structure 


In [ ]:
class CNNClassifier(nn.Module):
    def __init__(self):
        super(CNNClassifier, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 128 * 128, 512)
        self.fc2 = nn.Linear(512, 2)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(-1, 64 * 128 * 128)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

## Train Model

In [ ]:
# Check if a GPU is available and if not, default to CPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Training on {device}")

# Initialize the model, loss function, and optimizer
model = CNNClassifier().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Now you can use the model in your training loop
n_epochs = 10
for epoch in range(n_epochs):
    model.train()
    running_loss = 0.0
    for i, (inputs, masks, labels) in enumerate(data_loader):
        # Move data and labels to device
        inputs, labels = inputs.to(device), labels.to(device)
        mask = masks.to(device)
        inputs = masks.unsqueeze(1).float()
        optimizer.zero_grad()
        outputs = model(inputs).float()
        labels = labels.long()
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch + 1}/{n_epochs}, Loss: {running_loss / len(data_loader)}")
# Save the model
model_save_path = "./model.pth"
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")